# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import warnings

In [2]:
warnings.simplefilter('ignore')

In [3]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])


from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


import numpy as np
import pandas as pd

from sqlalchemy import create_engine
import re
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer , TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hider\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hider\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hider\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///data.db')
df = pd.read_sql("SELECT * FROM Messages", engine)

X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [8]:
X.head(3)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
Name: message, dtype: object

In [9]:
Y.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    '''
    INPUT
    text - a String
    
    the function will make the texte to a loxer case 
    then it will delete all the special caracters 
    after that it will split it to a list of words 
    finally it will lemetize the list of words 
    
    OUTPUT
    a list of tokenized word 
    '''
    text = text.lower()
    text = re.sub(r"[^a-zA-z0-9]"," ",text)
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    
    clean_tokens = []
    lemmatizer = WordNetLemmatizer()
    
    for w in words:
        clean_tok = lemmatizer.lemmatize(w , pos='v').strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [11]:
vect = CountVectorizer(tokenizer=tokenize)
X_vectorized = vect.fit_transform(X)

In [35]:
tfidf = TfidfTransformer()

<bound method BaseEstimator.get_params of TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)>

In [13]:
keys, values = [], []
for k, v in vect.vocabulary_.items():
    keys.append(k)
    values.append(v)

vocabulary = pd.DataFrame.from_dict({'words': keys, 'counts': values})

In [39]:
vocabulary.sample(15, random_state=72).sort_values('counts')

,words,counts
535,anymore,2791
19391,ari,3000
22684,consignment,6689
28682,fmoh,10585
26393,frontloaders,10940
25241,kulob,15298
22009,matchboxes,16880
28223,ngouboua,18604
14564,oruconnect,19541
13805,palomacv,19874


In [50]:
dic={}
for co in Y.columns:
   dic[co] = Y[co].sum()
dic

{'related': 19906,
 'request': 4474,
 'offer': 118,
 'aid_related': 10860,
 'medical_help': 2084,
 'medical_products': 1313,
 'search_and_rescue': 724,
 'security': 471,
 'military': 860,
 'child_alone': 0,
 'water': 1672,
 'food': 2923,
 'shelter': 2314,
 'clothing': 405,
 'money': 604,
 'missing_people': 298,
 'refugees': 875,
 'death': 1194,
 'other_aid': 3446,
 'infrastructure_related': 1705,
 'transport': 1201,
 'buildings': 1333,
 'electricity': 532,
 'tools': 159,
 'hospitals': 283,
 'shops': 120,
 'aid_centers': 309,
 'other_infrastructure': 1151,
 'weather_related': 7297,
 'floods': 2155,
 'storm': 2443,
 'fire': 282,
 'earthquake': 2455,
 'cold': 530,
 'other_weather': 1376,
 'direct_report': 5075}

In [12]:
hider = "i'm hider toula and i am studying At Sorbonne University"
print(tokenize(hider))

['hider', 'toula', 'study', 'sorbonne', 'university']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2, random_state = 45)

In [15]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None,
                                                                        criterion='gini',
                                                            

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
Y_pred = pipeline.predict(X_test)

In [17]:
def get_results(y_test, y_pred):
    '''
    INPUT
    y_test : the labels of the data set 
    y_pred : the labels of the predction on the data set
    
    calculate the averge of fscor , recall , pressision for each categorie 
    OUTPUT
    - fscore for each categorie 
    - recall for each categorie 
    - pressision for each categorie 
    '''
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    col = 0
    for categorie in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[categorie], y_pred[:,col], average='weighted')
        results.set_value(col+1, 'Category', categorie)
        results.set_value(col+1, 'f_score', f_score)
        results.set_value(col+1, 'precision', precision)
        results.set_value(col+1, 'recall', recall)
        col += 1
    print('avrege of  f_score:', results['f_score'].mean())
    print('avrege of precision:', results['precision'].mean())
    print('avrege of recall:', results['recall'].mean())
    return results

In [18]:
results = get_results(Y_test, Y_pred)
results

avrege of  f_score: 0.9304399423434854
avrege of precision: 0.9292884612094786
avrege of recall: 0.931761429120246


,Category,f_score,precision,recall
1,related,0.771262,0.770043,0.77257
2,request,0.847169,0.847238,0.8471
3,offer,0.992204,0.991037,0.993469
4,aid_related,0.713469,0.713351,0.7136
5,medical_help,0.903693,0.901086,0.906646
6,medical_products,0.940025,0.939068,0.94103
7,search_and_rescue,0.954343,0.951747,0.957357
8,security,0.968072,0.969005,0.967153
9,military,0.960656,0.9585,0.963888
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters =  {'tfidf__use_idf': (True,False)} 
cv = GridSearchCV(pipeline,parameters,cv=2,n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [21]:
Y_pred = cv.predict(X_test)

In [22]:
with open('model1.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])

In [24]:
parameters = {
    'vect__smooth_idf': [True,False],
}

cv = GridSearchCV(pipeline,param_grid=parameters,n_jobs=-1)

In [25]:
cv.fit(X_train, Y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                     

In [26]:
y_pred = cv.predict(X_test)

In [27]:
results = get_results(Y_test, y_pred)
results

avrege of  f_score: 0.9398941635791375
avrege of precision: 0.9398786828768078
avrege of recall: 0.9475444999359711


,Category,f_score,precision,recall
1,related,0.758802,0.772152,0.796005
2,request,0.881127,0.880927,0.889166
3,offer,0.99261,0.990417,0.994814
4,aid_related,0.746326,0.752199,0.751441
5,medical_help,0.919942,0.920368,0.932194
6,medical_products,0.953888,0.953644,0.960814
7,search_and_rescue,0.962222,0.961711,0.969458
8,security,0.974824,0.971889,0.981752
9,military,0.966537,0.965737,0.970803
10,child_alone,1,1,1


In [29]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x0000017DF1902318>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                      

### 9. Export your model as a pickle file

In [28]:
with open('model.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.